<div class="alert alert-info">
Ссылка для просмотра ноутбука в интерактивном режиме для использования гиперссылок и корректного отображения разметки:<br>
<a href='https://nbviewer.org/github/yulianikola/portfolio/blob/master/parsing_python/wildberries/wildberries.ipynb'>wildberries</a></div>

### Парсинг
#### Сайт wildberries.ru

#### Задача:
* Собрать информацию из карточек товаров по ссылкам из excel
* Искомая информация: артикул, наименование, бренд, страна производства, цена без скидки, цена со скидкой, число отзывов, рейтинг, кол-во продаж
* Консолидировать ежемесячную выгрузку в excel

<p id="0">
<ul type="square"><a href="#1"><li>Парсинг</li></a>
<a href="#2"><li>Обработка данных</li></a>
<a href="#3"><li>Консолидация данных</li></a>

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re

<p id="1"> 
<h4>Парсинг</h4>

Загружаем ссылки интересующих нас товаров:

In [2]:
links = pd.read_excel('C:\\python\\portfolio\\parsing_python\\wildberries\\data\\face_cream_links.xlsx', header = None)
links.head(2)

,0,1
0,Флакон с дозатором,https://www.wildberries.ru/catalog/5378143/det...
1,Банка,https://www.wildberries.ru/catalog/4803233/det...


In [3]:
links.columns = ['package', 'link']
links.head(2)

,package,link
0,Флакон с дозатором,https://www.wildberries.ru/catalog/5378143/det...
1,Банка,https://www.wildberries.ru/catalog/4803233/det...


In [4]:
links_list = links.link.values

Код сайта часто меняется. Чтобы не тратить время и лимит подключений к сайту, сделаем пробный прогон по парсингу нужной информации. По первой ссылке вся информация считалась корректно, кроме цен, так как товар не в наличии.

In [5]:
req = requests.get(links_list[0]) 
soup = BeautifulSoup(req.text, 'html.parser')
print(links_list[0])     
print('артикул', int(soup.find('p', class_='same-part-kt__article').select_one('span[data-link]').text))               
print('наименование', soup.find('h1', class_='same-part-kt__header').select_one('span[data-link*="goodsName"]').text) 
print('бренд', soup.find('h1', class_='same-part-kt__header').select_one('span[data-link*="brandName"]').text)
print('отзывы', int(re.search(r'\d+',soup.find('span', class_='same-part-kt__count-review').text.strip()).group(0)))
print('рейтинг', int(soup.select_one('span[data-link*="product^star"]').text.strip()))
print('страна', soup.find(string = 'Страна производства').find_parents('tr')[0].find('td').text)

scripts = soup.find_all('script')
keyword = 'preview_SpaWalletHistoryEntrypoint' 
for i in scripts: 
    if keyword in str(i):
        script_ind = scripts.index(i)

print('продажи', int( re.search(r'(?<=,"ordersCount":)\d+',str(scripts[script_ind])).group(0)))
print('цена изначальная', int( re.search(r'(?<=,"price":)\d+',str(scripts[script_ind])).group(0))) 
print('цена финальная', int( re.search(r'(?<=,"salePrice":)\d+',str(scripts[script_ind])).group(0)))

https://www.wildberries.ru/catalog/5378143/detail.aspx?targetUrl=XS
артикул 5378143
наименование Крем для лица мужской Ультра увлажнение, 115 мл / крем после бритья мужской / мужской крем для лица
бренд L Cosmetics
отзывы 194
рейтинг 5
страна Россия
продажи 7000


AttributeError: 'NoneType' object has no attribute 'group'

Проверим на второй ссылке, этот товар в наличии и цены извлекаются:

In [6]:
req = requests.get(links_list[1]) 
soup = BeautifulSoup(req.text, 'html.parser')
print(links_list[1])     

scripts = soup.find_all('script')
keyword = 'preview_SpaWalletHistoryEntrypoint' 
for i in scripts: 
    if keyword in str(i):
        script_ind = scripts.index(i)

print('цена изначальная', int( re.search(r'(?<=,"price":)\d+',str(scripts[script_ind])).group(0))) 
print('цена финальная', int( re.search(r'(?<=,"salePrice":)\d+',str(scripts[script_ind])).group(0)))

https://www.wildberries.ru/catalog/4803233/detail.aspx?targetUrl=XS
цена изначальная 248
цена финальная 181


Информация об артикуле, наименовании, бренде, числе отзывов, рейтинге по идее всегда должна быть в карточке товара. Поэтому в коде ниже для этих полей нет блока if-else NaN, так как ошибка будет означать изменение кода сайта. <br><br>
Информация о стране производства может отсутствовать в карточке товара. Так как мы предварительно проверили, не изменился ли код сайта, то в коде ошибку парсинга этого поля записываем как пропуск.<br><br>
Информация о ценах и продажах извлекается из скрипта/словаря. Продажи там по идее всегда должны быть, а вот информации о ценах может не быть, если товара нет в наличии. Поэтому по ценам тоже ошибку записываем как пропуск.<br><br>
За время использования данного кода парсинга код сайта менялся несколько раз. До какого-то момента из скрипта/словаря можно было извлечь точное количество продаж, а не округленное, которое отражается в самой карточке товара. Но теперь и в скрипте/словаре это кол-во только округленное. Найти, откуда извлечь точное кол-во, не удалось.

In [7]:
%%time

n_retries = 3
req = None

data = pd.DataFrame()
no_req = pd.DataFrame()

for link in links_list:

    # обращение к ссылкам через паузу во избежание ограничения запросов к сайту
    time.sleep(0.3)    

    # попытки достучаться до ссылки в случае не получения ответа(проблема с интернетом, сервером)
    for i in range(n_retries): 
        try:
            req = requests.get(link, timeout = 15)            
        except requests.Timeout as e:
            time.sleep(2**(i+1))
            req = None

    # проверка, если от ссылки нет ответа или, например, она больше не существует
    if req is None or req.status_code != 200: 
        # добавляем такую ссылку в отдельную таблицу, чтобы просмотреть позже
        no_req = pd.concat([no_req, pd.DataFrame([{'link':link}])]) 
        
    else:    
        soup = BeautifulSoup(req.text, 'html.parser')         

        # извлекаем артикул:        
        item_n = int(soup.find('p', class_='same-part-kt__article').select_one('span[data-link]').text) 

        # извлекаем наименование товара:                
        item_name = soup.find('h1', class_='same-part-kt__header').select_one('span[data-link*="goodsName"]').text 

        # извлекаем бренд:
        brand = soup.find('h1', class_='same-part-kt__header').select_one('span[data-link*="brandName"]').text 

        # извлекаем кол-во отзывов:
        review_n = int(re.search(r'\d+',soup.find('span', class_='same-part-kt__count-review').text.strip()).group(0))

        # извлекаем рейтинг:
        rating = int(soup.select_one('span[data-link*="product^star"]').text.strip())            

        # извлекаем страну производства:
        if soup.find(string='Страна производства'): 
            country = soup.find(string='Страна производства').find_parents('tr')[0].find('td').text
        else:
            country = np.NaN

        # извлекаем финальную цену:
        scripts = soup.find_all('script')
        keyword = 'preview_SpaWalletHistoryEntrypoint' 
        # найдем индекс нужного скрипта в списке по ключевому слову
        for i in scripts: 
            if keyword in str(i):
                script_ind = scripts.index(i)

        if re.search(r'(?<=,"salePrice":)\d+',str(scripts[script_ind])): 
            final_price = int( re.search(r'(?<=,"salePrice":)\d+',str(scripts[script_ind])).group(0)) 
        else:
            final_price = np.NaN                     

        # извлекаем изначальную цену:       
        if re.search(r'(?<=,"price":)\d+',str(scripts[script_ind])): 
            price = int( re.search(r'(?<=,"price":)\d+',str(scripts[script_ind])).group(0)) 
        else:
            price = np.NaN

        # извлекаем количество продаж:          
        sales = int( re.search(r'(?<=,"ordersCount":)\d+',str(scripts[script_ind])).group(0)) 

        #записываем в строку
        row = {'link': link, 'item_n':item_n, 'item_name':item_name, 'brand':brand, 'country':country,
               'price':price, 'final_price':final_price,
               'review_n':review_n, 'rating':rating,'sales':sales}

        data = pd.concat([data, pd.DataFrame([row])])

Wall time: 3min 49s


<h4><a href="#0">Наверх</a></h4>

<p id="2"> 
<h4>Обработка</h4>

Посмотрим, что за ссылки не ответили на запрос. Если эти страницы больше не существуют, то можно убрать их из списка ссылок, чтоб в следующий раз не тратить на них лимит запросов.

In [10]:
no_req.to_excel('C:\\python\\portfolio\\parsing_python\\wildberries\\data\\no_req_25_12.xlsx')

In [11]:
no_req['ind'] = '1'
no_req.head(2)

,link,ind
0,https://www.wildberries.ru/catalog/28030979/de...,1
0,https://www.wildberries.ru/catalog/28030469/de...,1


In [12]:
links_new = links.merge(no_req, how = 'left', on = 'link')
links_new = links_new[links_new.ind.isna()].drop('ind', axis = 1)
links_new.to_excel('C:\\python\\portfolio\\parsing_python\\wildberries\\data\\face_cream_links_new.xlsx')

Оставляем пока индексный столбец с нулями, дальше после объединения это исправится.

In [13]:
data.head(2)

,link,item_n,item_name,brand,country,price,final_price,review_n,rating,sales
0,https://www.wildberries.ru/catalog/5378143/det...,5378143,"Крем для лица мужской Ультра увлажнение, 115 м...",L Cosmetics,Россия,NaN,NaN,194,5,7000
0,https://www.wildberries.ru/catalog/4803233/det...,4803233,Крем для лица мужской Men интенсивно увлажняющ...,Nivea,Германия,248.0,181.0,782,5,33300


Проверим, что в "обязательных" полях item_n, item_name, brand, review_n, rating, sales нет пропусков. Пропуски  могут быть по полям: country, price, final_price.

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252 entries, 0 to 0
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   link         252 non-null    object 
 1   item_n       252 non-null    int64  
 2   item_name    252 non-null    object 
 3   brand        252 non-null    object 
 4   country      235 non-null    object 
 5   price        158 non-null    float64
 6   final_price  158 non-null    float64
 7   review_n     252 non-null    int64  
 8   rating       252 non-null    int64  
 9   sales        252 non-null    int64  
dtypes: float64(2), int64(4), object(4)
memory usage: 21.7+ KB


Подтягиваем информацию об упаковке товара из файла со ссылками:

In [15]:
data_full = data.merge(links, how = 'left', on = 'link')
data_full.head(2)

,link,item_n,item_name,brand,country,price,final_price,review_n,rating,sales,package
0,https://www.wildberries.ru/catalog/5378143/det...,5378143,"Крем для лица мужской Ультра увлажнение, 115 м...",L Cosmetics,Россия,NaN,NaN,194,5,7000,Флакон с дозатором
1,https://www.wildberries.ru/catalog/4803233/det...,4803233,Крем для лица мужской Men интенсивно увлажняющ...,Nivea,Германия,248.0,181.0,782,5,33300,Банка


Добавляем дату выгрузки:

In [16]:
data_full['timestamp'] = pd.to_datetime('today')
data_full['date'] = data_full['timestamp'].dt.strftime('%d.%m.%Y')
data_full['month'] = data_full['timestamp'].dt.month
data_full['year'] = data_full['timestamp'].dt.year

Финальный датафрейм:

In [17]:
data_full.head(2)

,link,item_n,item_name,brand,country,price,final_price,review_n,rating,sales,package,timestamp,date,month,year
0,https://www.wildberries.ru/catalog/5378143/det...,5378143,"Крем для лица мужской Ультра увлажнение, 115 м...",L Cosmetics,Россия,NaN,NaN,194,5,7000,Флакон с дозатором,2021-12-25 17:35:18.988451,25.12.2021,12,2021
1,https://www.wildberries.ru/catalog/4803233/det...,4803233,Крем для лица мужской Men интенсивно увлажняющ...,Nivea,Германия,248.0,181.0,782,5,33300,Банка,2021-12-25 17:35:18.988451,25.12.2021,12,2021


Записываем датафрейм в excel:

In [18]:
data_full.to_excel('C:\\python\\portfolio\\parsing_python\\wildberries\\data\\face_cream_info_25_12.xlsx')

<h4><a href="#0">Наверх</a></h4>

<p id="3"> 
<h4>Консолидация данных</h4>

 Добавление данных за новый месяц к аккумулированным данным.

In [19]:
data_time = pd.read_excel('C:\\python\\portfolio\\parsing_python\\wildberries\\data\\face_cream_info_time.xlsx', index_col  = 0)

In [20]:
data_time.head(2)

,link,item_n,item_name,brand,country,price,final_price,review_n,rating,sales,package,date,month,year,sales_delta,review_delta
0,https://www.wildberries.ru/catalog/5378143/det...,5378143.0,"Крем для лица Ультра увлажнение, 115 мл",L Cosmetics,Россия,NaN,NaN,169.0,5.0,6259.0,Флакон с дозатором,24.05.2021,5,2021,NaN,NaN
1,https://www.wildberries.ru/catalog/4803233/det...,4803233.0,Крем для лица мужской Men интенсивно увлажняющ...,Nivea,Германия,228.0,158.0,385.0,5.0,20011.0,Банка,24.05.2021,5,2021,NaN,NaN


К файлу с накопленными данными присоединяем данные за новый месяц:

In [21]:
data_time = pd.concat([data_time, data_full.drop('timestamp', axis = 1)], ignore_index = True)

In [22]:
data_time.tail(2)

,link,item_n,item_name,brand,country,price,final_price,review_n,rating,sales,package,date,month,year,sales_delta,review_delta
1790,https://www.wildberries.ru/catalog/6952079/det...,6952079.0,Подарочный набор для бритья с тестером (крем д...,MORGAN'S,Соединенное Королевство,NaN,NaN,0.0,0.0,6.0,Набор,25.12.2021,12,2021,NaN,NaN
1791,https://www.wildberries.ru/catalog/21007993/de...,21007993.0,Бальзам после бритья Man Sport & Sauna 75 мл.,Dermosil,Финляндия,774.0,619.0,1.0,5.0,20.0,Туба,25.12.2021,12,2021,NaN,NaN


Так как продажи в карточке товара являются показателем запаса (накопленное кол-во), то для анализа динамики рассчитаем показатель потока (разница между текущим и предыдущим периодом). То же самое для кол-ва отзывов.

In [23]:
g = data_time.groupby('link')
data_time['sales_delta'] = g['sales'].apply(lambda x: x - x.shift())

In [24]:
data_time['review_delta'] = g['review_n'].apply(lambda x: x - x.shift())

Перезаписываем файл с накопленными данными:

In [25]:
data_time.to_excel('C:\\python\\portfolio\\parsing_python\\wildberries\\data\\face_cream_info_time.xlsx')

<h4><a href="#0">Наверх</a></h4>